In [1]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.svm import SVC
from nltk.corpus import stopwords

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
path_2 = '/Users/raisaurabh04/OneDrive/GreyAtom/Practice Dataset/full_complaint_nlp_path_2.csv'

path = '/Users/raisaurabh04/OneDrive/GreyAtom/Practice Dataset/new_complaint_nlp_path.csv'

full_data = pd.read_csv(path)

data = full_data[['Consumer complaint narrative', 'Product']]

data.rename(columns={'Product' : 'y', 'Consumer complaint narrative' : 'X'}, inplace=True)

data['X'][1]

data['y'].unique()

/Users/raisaurabh04/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


'When my loan was switched over to Navient i was never told that i had a deliquint balance because with XXXX i did not. When going to purchase a vehicle i discovered my credit score had been dropped from the XXXX into the XXXX. I have been faithful at paying my student loan. I was told that Navient was the company i had delinquency with. I contacted Navient to resolve this issue you and kept being told to just contact the credit bureaus and expalin the situation and maybe they could help me. I was so angry that i just hurried and paid the balance off and then after tried to dispute the delinquency with the credit bureaus. I have had so much trouble bringing my credit score back up.'

array(['Mortgage', 'Student loan', 'Credit card or prepaid card',
       'Credit card', 'Debt collection', 'Credit reporting',
       'Credit reporting, credit repair services, or other personal consumer reports',
       'Bank account or service', 'Consumer Loan', 'Money transfers',
       'Vehicle loan or lease',
       'Money transfer, virtual currency, or money service',
       'Checking or savings account', 'Payday loan',
       'Payday loan, title loan, or personal loan',
       'Other financial service', 'Prepaid card'], dtype=object)

In [4]:
data.dropna(inplace=True)

first_complaint = data.iloc[0, 0]

bag_of_words_1 = first_complaint.split()

bag_of_words_2 = word_tokenize(first_complaint)

/Users/raisaurabh04/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [5]:
bag_of_words_1 in bag_of_words_2

False

In [6]:
bag_of_words_1

['When',
 'my',
 'loan',
 'was',
 'switched',
 'over',
 'to',
 'Navient',
 'i',
 'was',
 'never',
 'told',
 'that',
 'i',
 'had',
 'a',
 'deliquint',
 'balance',
 'because',
 'with',
 'XXXX',
 'i',
 'did',
 'not.',
 'When',
 'going',
 'to',
 'purchase',
 'a',
 'vehicle',
 'i',
 'discovered',
 'my',
 'credit',
 'score',
 'had',
 'been',
 'dropped',
 'from',
 'the',
 'XXXX',
 'into',
 'the',
 'XXXX.',
 'I',
 'have',
 'been',
 'faithful',
 'at',
 'paying',
 'my',
 'student',
 'loan.',
 'I',
 'was',
 'told',
 'that',
 'Navient',
 'was',
 'the',
 'company',
 'i',
 'had',
 'delinquency',
 'with.',
 'I',
 'contacted',
 'Navient',
 'to',
 'resolve',
 'this',
 'issue',
 'you',
 'and',
 'kept',
 'being',
 'told',
 'to',
 'just',
 'contact',
 'the',
 'credit',
 'bureaus',
 'and',
 'expalin',
 'the',
 'situation',
 'and',
 'maybe',
 'they',
 'could',
 'help',
 'me.',
 'I',
 'was',
 'so',
 'angry',
 'that',
 'i',
 'just',
 'hurried',
 'and',
 'paid',
 'the',
 'balance',
 'off',
 'and',
 'then',
 'a

In [7]:
bag_of_words_2

['When',
 'my',
 'loan',
 'was',
 'switched',
 'over',
 'to',
 'Navient',
 'i',
 'was',
 'never',
 'told',
 'that',
 'i',
 'had',
 'a',
 'deliquint',
 'balance',
 'because',
 'with',
 'XXXX',
 'i',
 'did',
 'not',
 '.',
 'When',
 'going',
 'to',
 'purchase',
 'a',
 'vehicle',
 'i',
 'discovered',
 'my',
 'credit',
 'score',
 'had',
 'been',
 'dropped',
 'from',
 'the',
 'XXXX',
 'into',
 'the',
 'XXXX',
 '.',
 'I',
 'have',
 'been',
 'faithful',
 'at',
 'paying',
 'my',
 'student',
 'loan',
 '.',
 'I',
 'was',
 'told',
 'that',
 'Navient',
 'was',
 'the',
 'company',
 'i',
 'had',
 'delinquency',
 'with',
 '.',
 'I',
 'contacted',
 'Navient',
 'to',
 'resolve',
 'this',
 'issue',
 'you',
 'and',
 'kept',
 'being',
 'told',
 'to',
 'just',
 'contact',
 'the',
 'credit',
 'bureaus',
 'and',
 'expalin',
 'the',
 'situation',
 'and',
 'maybe',
 'they',
 'could',
 'help',
 'me',
 '.',
 'I',
 'was',
 'so',
 'angry',
 'that',
 'i',
 'just',
 'hurried',
 'and',
 'paid',
 'the',
 'balance',
 'o

In [8]:
list_of_sentences = sent_tokenize(first_complaint)

first_complaint_lower = first_complaint.lower()

bag_of_words_lower = word_tokenize(first_complaint_lower)

In [9]:
count_vectorizer = Counter(bag_of_words_lower)






<h3>For X</h3>

- Store the "X" column of dataframe 'data'in a new dataframe 'all_text'(i.e. instead of data["X"] use data[["X"]])

- Convert the values of X column of all_text into lowercase using "lower()" method.

- Initialise a "CountVectorizer()" object and store it in 'cv'

- "fit_transform()" the 'cv' on all_text[X] column and store the result in 'vector'

- Convert 'vector' into an array using "toarray()" method and store the result in a new variable 'X'

<h3>For y</h3>

- Store the "y" column of dataframe 'data'in a new dataframe 'labels'(i.e. instead of data["y"] use data[["y"]])

- We need to label encode the values. Therefore initialise a "LabelEncoder()" object and store it in 'le'

- fit_transform method the 'le' on "labels[y]" and store the results back in labels[y].

  If we now, include all the 335 rows and vectorize them and label it as the X, and the corresponding y labels, we can train a classification algorithm on the same, and figure out the accuracy.

<h3>Model building</h3>
- Split 'X' and 'labels["y"]' into X_train,X_test,y_train,y_test using train_test_split() function. Use test_size = 0.4 and random_state = 42

- Initialise a logistic regression model with LogisticRegression() having random_state=42 and save it to a variable called 'log_reg'.

- Fit the model on the training data 'X_train' and 'y_train' using the 'fit()' method.

- Find out the accuracy score between X_test and 'y_test' using the 'score()' method and save it in a variable called 'acc'

In [10]:
all_text = data[['X']]

all_text = all_text.applymap(lambda x : x.lower())
all_text.head()
cv = CountVectorizer()

vector = cv.fit_transform(all_text['X'])

X = vector.toarray()

,X
1,when my loan was switched over to navient i wa...
2,i tried to sign up for a spending monitoring p...
7,"my mortgage is with bb & t bank, recently i ha..."
13,the entire lending experience with citizens ba...
14,my credit score has gone down xxxx points in t...


In [11]:
labels = data[['y']]

le = LabelEncoder()

labels['y'] = le.fit_transform(labels['y'])

/Users/raisaurabh04/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [12]:
X.shape, labels['y'].shape

((335, 4134), (335,))

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, labels["y"], test_size=0.40, random_state=42)

log_reg = LogisticRegression(random_state=42)

log_reg.fit(X_train, y_train)

acc = log_reg.score(X_test, y_test)

/Users/raisaurabh04/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/raisaurabh04/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)







<h3>Stopword handling</h3>

- Initialise a "CountVectorizer()" object with parameter stop_words= "english" and store it in 'cv_stop'
- Apply the "fit_transform()" method of 'cv' on X column and store the result in 'vector_stop'
- Convert 'vector_stop' into an array using "toarray()" method and store the result in a new variable 'X_stop'
<h3>Model building</h3>

- Split 'X_stop' and 'labels["y"]' into X_train,X_test,y_train,y_test using train_test_split() function. Use test_size = 0.4 and random_state = 42
- Initialise a logistic regression model with LogisticRegression() having random_state=42 and save it to a variable called 'log_reg'.
- Fit the model on the training data 'X_train' and 'y_train' using the 'fit()' method.
- Find out the accuracy score between X_test and 'y_test' using the 'score()' method and save it in a variable called 'stop_acc'





In [14]:
cv_stop = CountVectorizer(stop_words='english')

vector_stop = cv_stop.fit_transform(all_text['X'])

X_stop = vector_stop.toarray()

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_stop, labels['y'], test_size=0.4, random_state=42)

log_reg = LogisticRegression(random_state=42)

log_reg.fit(X_train, y_train)

stop_acc = log_reg.score(X_test, y_test)

/Users/raisaurabh04/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/raisaurabh04/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

<h3>TF-IDF scoring</h3>

- Initialise a "TfidfVectorizer()" object with parameter stop_words= "english" and store it in 'tfidf'
- Apply the "fit_transform()" method of 'tfidf' on X column and store the result in 'vector_tfidf'
- Convert 'vector_tfidf' into an array using "toarray()" method and store the result in a new variable 'X_tfidf'

<h3>Model building</h3>

- Split 'X_tfidf' and 'labels["y"]' into X_train,X_test,y_train,y_test using train_test_split() function. Use test_size = 0.4 and random_state = 42
- Initialise a logistic regression model with LogisticRegression() having random_state=42 and save it to a variable called 'log_reg'.
- Fit the model on the training data 'X_train' and 'y_train' using the 'fit()' method.
- Find out the accuracy score between X_test and 'y_test' using the 'score()' method and save it in a variable called 'tfidf_acc'





In [16]:
tfidf = TfidfVectorizer(stop_words='english')

vector_tfidf = tfidf.fit_transform(all_text['X'])

X_tfidf = vector_tfidf.toarray()

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, labels['y'], test_size=0.4, random_state=42)

log_reg = LogisticRegression(random_state=42)

log_reg.fit(X_train, y_train)

tfidf_acc = log_reg.score(X_test, y_test)

/Users/raisaurabh04/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/raisaurabh04/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)






<h3>Data Loading</h3>

- Load the dataset from 'path_2'(given) using the read_csv() method from pandas and store it in 'data'.
- Subset the dataframe 'data' to only include "Consumer complaint narrative" and "Product" and store this dataframe subset back in 'data'
- Rename the column "Consumer complaint narrative" to "X" and "Product" to "y" by assigning ["X","y"] to data.columns
- Drop nan values from data using "dropna()" and save it back to 'data'

<h3>For X</h3>

- Store the "X" column of dataframe 'data'in a new dataframe 'all_text'(i.e. instead of data["X"] use data[["X"]])
- Convert the values of X column of all_text into lowercase using "lower()" method.

<h3>TF-IDF vectoriser</h3>

- Initialise a "TfidfVectorizer()" object with parameter stop_words= "english" and store it in 'tfidf'
- Apply the "fit_transform()" method of 'tfidf' on X column and store the result in 'vector_tfidf'
- Convert 'vector_tfidf' into an array using "toarray()" method and store the result in a new variable 'X_tfidf'

<h3>For y</h3>

- Store the "y" column of dataframe 'data'in a new dataframe 'labels'(i.e. instead of data["y"] use data[["y"]])
- We need to label encode the values. Therefore initialise a "LabelEncoder()" object and store it in 'le'
- Use the fit_transform method of 'le' on column "y" of 'labels' and store the results back in y column

<h3>Model building</h3>

- Split 'X_tfidf' and 'labels["y"]' into X_train,X_test,y_train,y_test using train_test_split() function. Use test_size = 0.4 and random_state = 42
- Initialise a naive bayes model with MultinomialNB() having and save it to a variable called 'nb'
- Fit the model on the training data 'X_train' and 'y_train' using the 'fit()' method.
- Find out the accuracy score between X_test and 'y_test' using the 'score()' method and save it in a variable called 'nb_acc'









In [18]:
data = pd.read_csv(path_2)

data = data[['Consumer complaint narrative', 'Product']]

data.rename(columns={'Product' : 'y', 'Consumer complaint narrative' : 'X'}, inplace=True)

data.dropna(inplace=True)

In [19]:
all_text = data[['X']]

all_text = all_text.applymap(lambda x : x.lower())
all_text.head()

,X
1,when my loan was switched over to navient i wa...
2,i tried to sign up for a spending monitoring p...
7,"my mortgage is with bb & t bank, recently i ha..."
13,the entire lending experience with citizens ba...
14,my credit score has gone down xxxx points in t...


In [20]:
tfidf = TfidfVectorizer(stop_words='english')

vector_tfidf = tfidf.fit_transform(all_text['X'])

X_tfidf = vector_tfidf.toarray()

In [21]:
labels = data[['y']]

le = LabelEncoder()

labels['y'] = le.fit_transform(labels['y'])

/Users/raisaurabh04/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, labels['y'], test_size=0.4, random_state=42)

nb = MultinomialNB()

nb.fit(X_train, y_train)

nb_acc = nb.score(X_test, y_test)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [23]:
nb_acc

0.42461964038727523





<h3>Sampling</h3>

- In this task, we will oversample the data and try to implement Naive Bayes classifier again.

<h3>Instructions</h3>

- Initialise a RandomOverSampler() object with random_state=0 and save it to a variable called 'ros'.
- Using fit_sample() method of 'ros', undersample 'X_train' and 'y_train' and store the new samples in variables 'X_ros' and 'y_ros'.
- Initialise a naive bayes model and save it to a variable called 'nb'.
- Fit the model on the training data 'X_ros' and 'y_ros' using the 'fit()' method.
- Find out the accuracy score between X_test and 'y_test' using the 'score()' method and save it in a variable called 'ros_score'





In [24]:
ros = RandomOverSampler(random_state=0)

X_ros, y_ros = ros.fit_sample(X_train, y_train)

nb = MultinomialNB()

nb.fit(X_ros, y_ros)

ros_score = nb.score(X_test, y_test)

ros_score

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

0.6016597510373444





<h3>Linear SVC</h3>

- Let's use a Linear SVC as our last algorithm, to test the results.
- Initialise a support vector model with SVC() with random_state=0& kernel="linear" and save it to a variable called 'svc'.
- Fit the model on the training data 'X_ros' and 'y_ros' using the 'fit()' method.
- Find out the accuracy score between X_test and 'y_test' using the 'score()' method and save it in a variable called 'svc_score'


In [25]:
svc = SVC(random_state=0, kernel='linear')

In [26]:
svc.fit(X_ros, y_ros)

svc_score = svc.score(X_test, y_test)

svc_score

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=0,
    shrinking=True, tol=0.001, verbose=False)

0.648686030428769




# Assessment









First task for you guys is to load data, take only the relevant columns and observing the distribution of the class labels.

<h3>Instructions</h3>

- Load data using path variable using .read_csv() method of pandas. Save it as news
- All the columns are not relevant and you will be going forward with TITLE (title of resource) and CATEGORY (class label). Select only these two features and store it as news
- Now, get the class distribution of class lables. You can get it using .value_counts() method on CATEGORY column of news dataframe. Store it inside dist variable
- Print out dist and news.head() to observe the class-wise distributions and the first observations respectively

In [33]:
path = '/Users/raisaurabh04/OneDrive/GreyAtom/NLP_Assessment.zip'

news = pd.read_csv(path)

news = news[['TITLE', 'CATEGORY']]

dist = news['CATEGORY'].value_counts()

news.head()

dist

,TITLE,CATEGORY
0,"Fed official says weak data caused by weather,...",b
1,Fed's Charles Plosser sees high bar for change...,b
2,US open: Stocks fall after Fed official hints ...,b
3,"Fed risks falling 'behind the curve', Charles ...",b
4,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,b


e    152469
b    115967
t    108344
m     45639
Name: CATEGORY, dtype: int64



<h3>Instructions</h3>

- Initialize stopwords as stop with set(stopwords.words('english'))

- To retain only alphabets for every instance, use a lambda function in combination with .apply() method that does so. The function that you will be applying to every instance (supposing the instance is row) will be re.sub("[^a-zA-Z]", " ",x). Remember this operation should be carried out on TITLE column only

- Next use lambda function and .apply() method to first convert the instances to lowercase (using .lower()) and then tokenize (using .split()). Remember this operation should be carried out on TITLE column only

- Now time to remove stopwords from every instance. Again using a combination of lambda function and .apply() method retain only words which are in that instance but not in stop. You can take the help of a list comprehension to achieve it. For ex: [i for i in x if i not in y]. Remember this operation should be carried out on TITLE column only

- The steps mentioned above gives a list for every instance across TITLE column. Join the list elements into a single sentence using ' '.join() method of lists. Use both lambda function and .apply() method for it.

- Finally split into train and test using train_test_split function where feature is news["TITLE"], target is news["CATEGORY"], test size is 20% and random state is 3. Save the resultant variables as X_train, X_test, Y_train and Y_test

In [34]:
stop = set(stopwords.words('english'))

news['TITLE'] = news['TITLE'].map(lambda x : re.sub('[^a-zA-Z]', ' ', x).lower())

In [35]:
news['TITLE'] = news['TITLE'].map(lambda x : x.split())        

In [36]:
news['TITLE'].apply(lambda x : [i for i in x if i not in stop])

0         [fed, official, says, weak, data, caused, weat...
1         [fed, charles, plosser, sees, high, bar, chang...
2         [us, open, stocks, fall, fed, official, hints,...
3         [fed, risks, falling, behind, curve, charles, ...
4         [fed, plosser, nasty, weather, curbed, job, gr...
5           [plosser, fed, may, accelerate, tapering, pace]
6                    [fed, plosser, taper, pace, may, slow]
7         [fed, plosser, expects, us, unemployment, fall...
8         [us, jobs, growth, last, month, hit, weather, ...
9         [ecb, unlikely, end, sterilisation, smp, purch...
10        [ecb, unlikely, end, sterilization, smp, purch...
11              [eu, half, baked, bank, union, could, work]
12         [europe, reaches, crunch, point, banking, union]
13        [ecb, focus, stronger, euro, drowns, ecb, mess...
14               [eu, aims, deal, tackling, failing, banks]
15            [forex, pound, drops, one, month, lows, euro]
16        [noyer, says, strong, euro, cr

In [28]:
news['TITLE'] = news['TITLE'].apply(lambda x : [i for i in x if i not in stop])

news['TITLE'] = news['TITLE'].apply(lambda x : " ".join(x))        

X = news['TITLE']

y = news['CATEGORY']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

In [32]:
news

,TITLE,CATEGORY
0,fed official says weak data caused weather slo...,b
1,fed charles plosser sees high bar change pace ...,b
2,us open stocks fall fed official hints acceler...,b
3,fed risks falling behind curve charles plosser...,b
4,fed plosser nasty weather curbed job growth,b
5,plosser fed may accelerate tapering pace,b
6,fed plosser taper pace may slow,b
7,fed plosser expects us unemployment fall end,b
8,us jobs growth last month hit weather fed pres...,b
9,ecb unlikely end sterilisation smp purchases t...,b




<h3>Instructions</h3>

- Initialize Bag-of-words vectorizer using CountVectorizer() and TF-IDF vectorizer using TfidfVectorizer(ngram_range=(1,3)). Save them as count_vectorizer and tfidf_vectorizer respectively

- Next thing to do is fit each vectorizer on training and test features with text data and transform them to vectors.

    * First fit and transform data with count_vectorizer on X_train using .fit_transform(X_train) method of count_vectorizer and save it as X_train_count

    * Use this fitted version of count_vectorizer on X_test and transform X_test with .transform(X_test) method of count_vectorizer. Save it as X_test_count

    * Similarly repeat the previous two steps with tfidf_vectorizer and save the transformed training feature as X_train_tfidf and transformed test feature as X_test_tfidf


In [29]:
count_vectorizer = CountVectorizer()
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3))

X_train_count = count_vectorizer.fit_transform(X_train)

X_test_count = count_vectorizer.transform(X_test)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

X_test_tfidf = tfidf_vectorizer.transform(X_test)

<h3>Instructions</h3>

- First initialize two Multinomial Naive Bayes classifiers with MultinomialNB() and save them as nb_1 and nb_2. The reason for initializing two classifiers is because you will be training and testing on both Bag-of-words and TF-IDF transformed training data

- Fit nb_1 on X_train_count and Y_train using .fit() method

- Fit nb_2 on X_train_tfidf and Y_train using .fit() method

- Find the accuracy with Bag-of-words approach using accuracy_score(nb_1.predict(X_test_count), Y_test) and save it as acc_count_nb

- Similarly find the accuracy for the TF-IDF approach (only difference is the classifer is nb_2) and save it as acc_tfidf_nb

- Print out acc_count_nb and acc_tfidf_nb to check which version performs better for with Multinomial Naive Bayes as classifer

In [30]:
nb_1, nb_2 = MultinomialNB(), MultinomialNB()

nb_1.fit(X_train_count, y_train)

nb_2.fit(X_train_tfidf, y_train)

acc_count_nb = accuracy_score(y_test, nb_1.predict(X_test_count))

acc_count_nb

acc_tfidf_nb = accuracy_score(y_test, nb_2.predict(X_test_tfidf))

acc_tfidf_nb

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

0.9268618910089484

0.9323895648880262



<h3>Instructions</h3>

- First initialize two classifiers with OneVsRestClassifier(LogisticRegression(random_state=10)) and save them as logreg_1 and logreg_2. The reason for initializing two classifiers is because you will be training and testing on both Bag-of-words and TF-IDF transformed training data
- Fit logreg_1 on X_train_count and Y_train using .fit() method
- Fit logreg_2 on X_train_tfidf and Y_train using .fit() method
- Find the accuracy with Bag-of-words approach using accuracy_score(logreg_1.predict(X_test_count), Y_test) and save it as acc_count_logreg
- Similarly find the accuracy for the TF-IDF approach (only difference is the classifer is logreg_2) and save it as acc_tfidf_logreg
- Print out acc_count_logreg and acc_tfidf_logreg to check which version performs better for with Multinomial Naive Bayes as classifer

In [31]:
logreg_1, logreg_2 = OneVsRestClassifier(LogisticRegression(random_state=10)), OneVsRestClassifier(LogisticRegression(random_state=10))

logreg_1.fit(X_train_count, y_train)

logreg_2.fit(X_train_tfidf, y_train)

acc_count_logreg = accuracy_score(y_test, logreg_1.predict(X_test_count))

acc_count_logreg

acc_tfidf_logreg = accuracy_score(y_test, logreg_2.predict(X_test_tfidf))

acc_tfidf_logreg

/Users/raisaurabh04/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=10, solver='warn',
                                                 tol=0.0001, verbose=0,
                                                 warm_start=False),
                    n_jobs=None)

/Users/raisaurabh04/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=10, solver='warn',
                                                 tol=0.0001, verbose=0,
                                                 warm_start=False),
                    n_jobs=None)

0.9464632356422518

0.9428649211685053